import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
#rplc imprt data , (1,cls_num+1)<> (0,cls_num) ,( [i-1, <<>> [i ) , Ntwrk/CNN1/CNN2  , ClsPrcnt , AugPrcnt 

In [2]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
rate=480
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=MixedShapesSmallTrain(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (100, 1025)
first_test_shape= (2425, 1025)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [20 20 20 20 20]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -2.8941765
first_train_sample=
 [1.        1.1309687 1.1204832 ... 1.0931087 1.1025334 1.1120469]
1 20	2 20	3 20	4 20	5 20	

In [4]:
np.shape(ecg)

(100, 481)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

each class and its windows =
1 20	2 20	3 20	4 20	5 20	

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,5), int) 
rslts=np.array([])
for X in range(1,5):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([82,84,83,84,83,83])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            
            scors0=np.array([88,93,79,68,98])                       #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            Cls_amnt=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*Cls_amnt[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(MxMn20fscr*np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            rprt=np.empty((0,3),float)
            print('aug_amnt=\n',aug_amnt)

            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-5  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=1            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort),axis=0)
                    ''''if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort),axis=0)'''
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                if min(y_train)==1:
                    y_train=y_train-1
                    y_test=y_test-1
                    y_valid =y_valid-1
                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                #VGG16_1D

                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :

                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data

                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 21
aug_amnt=
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 1  average= [83 83 83]  max= [84 84 84]  var= [1 1 1]
aug_amnt=
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 2  average= [82 82 82]  max= [85 85 85]  var= [4 5 6]
aug_amnt=
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 6  average= [83 83 83]  max= [84 84 84]  var= [1 1 1]
aug_amnt=
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 1  alpha 2  Beta 1  average= [83 83 83]  max= [84 84 84]  var= [0 0 0]
aug_amnt=
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 1  alpha 2

76/76 [==============================] - 0s 2ms/step
rslts= X 3  alpha 3  Beta 6  average= [83 82 83]  max= [83 83 83]  var= [0 0 0]
aug_amnt=
 [[3 1 3 1 1]
 [3 1 3 1 1]
 [3 1 3 1 1]
 [4 2 4 2 2]
 [3 1 3 1 1]]
76/76 [==============================] - 0s 2ms/step
rslts= X 4  alpha 1  Beta 1  average= [84 83 83]  max= [85 84 84]  var= [0 0 0]
aug_amnt=
 [[4 1 4 1 1]
 [4 1 4 1 1]
 [4 1 4 1 1]
 [5 1 5 1 1]
 [4 0 4 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 4  alpha 1  Beta 2  average= [83 83 83]  max= [84 84 83]  var= [0 0 0]
aug_amnt=
 [[5 0 5 0 0]
 [5 0 5 0 0]
 [5 0 5 0 0]
 [7 0 7 0 0]
 [5 0 5 0 0]]
76/76 [==============================] - 0s 2ms/step
rslts= X 4  alpha 1  Beta 6  average= [84 83 84]  max= [84 84 84]  var= [0 0 0]
aug_amnt=
 [[4 1 4 1 1]
 [3 1 3 1 1]
 [4 2 4 2 2]
 [5 2 5 2 2]
 [3 1 3 1 1]]
76/76 [==============================] - 0s 2ms/step
rslts= X 4  alpha 2  Beta 1  average= [83 83 83]  max= [84 84 84]  var= [0 0 0]
aug_amnt=
 [[4 1 4 1 1]
 [4

In [7]:
rslts

array(['X 1  alpha 1  Beta 1  average= [83 83 83]  max= [84 84 84]  var= [1 1 1]',
       'X 1  alpha 1  Beta 2  average= [82 82 82]  max= [85 85 85]  var= [4 5 6]',
       'X 1  alpha 1  Beta 6  average= [83 83 83]  max= [84 84 84]  var= [1 1 1]',
       'X 1  alpha 2  Beta 1  average= [83 83 83]  max= [84 84 84]  var= [0 0 0]',
       'X 1  alpha 2  Beta 2  average= [83 83 83]  max= [84 84 84]  var= [0 0 0]',
       'X 1  alpha 2  Beta 6  average= [83 83 83]  max= [84 83 83]  var= [0 0 0]',
       'X 1  alpha 3  Beta 1  average= [83 82 82]  max= [83 83 83]  var= [0 0 0]',
       'X 1  alpha 3  Beta 2  average= [83 83 83]  max= [84 84 84]  var= [0 0 0]',
       'X 1  alpha 3  Beta 6  average= [84 84 84]  max= [84 84 84]  var= [0 0 0]',
       'X 2  alpha 1  Beta 1  average= [84 84 84]  max= [84 84 84]  var= [0 0 0]',
       'X 2  alpha 1  Beta 2  average= [84 83 83]  max= [84 84 84]  var= [0 0 0]',
       'X 2  alpha 1  Beta 6  average= [82 82 82]  max= [83 83 83]  var= [2 2 2]',
    